## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
# 讀取資料集
diabetes = datasets.load_breast_cancer()

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(diabetes.data, diabetes.target,
                                                    test_size=0.25, random_state=42)

# 建立模型
clf = GradientBoostingClassifier()

In [3]:
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print('Acuuracy:',metrics.accuracy_score(y_test, y_pred))

Acuuracy: 0.958041958041958


In [26]:
# 設定要訓練的超參數組合
n_estimators = [50, 100, 150]
max_depth = [1, 3, 5]
learning_rate = [0.1, 0.2, 0.3, 1.0]

#產生參數表格(字典)
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="accuracy", n_jobs=-1, verbose=1, cv=3)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 36 種參數組合，總共要 train 108 次模型

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:    9.7s finished


In [27]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.974178 using {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 150}


In [30]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingClassifier(learning_rate=grid_result.best_params_['learning_rate'],
                                           max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

In [31]:
# 調整參數後約可降至 0.05 的 MSE
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.972027972027972
